In [5]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# pip3 install lxml

# load secrets
load_dotenv()

True

In [13]:
def get_historical_data(symbol, lookback_days=90):
    iex_key = os.getenv("IEX_API_KEY") 
    historical_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/chart/{lookback_days}d?token={iex_key}'

    historical_data = requests.get(historical_url).json()
    df = pd.DataFrame(historical_data)
    return df[['date', 'high', 'low', 'close']]

# Fetch historical data
ticker = 'PLTR'
historical_df = get_historical_data(ticker)

# Display the historical data
# print(historical_df)

In [14]:
# display html version of df
historical_df.tail(5)

,date,high,low,close
55,2024-01-11,16.93,16.17,16.68
56,2024-01-12,17.06,16.62,16.76
57,2024-01-16,16.74,16.29,16.53
58,2024-01-17,16.41,16.05,16.39
59,2024-01-18,16.81,16.10,16.40


In [27]:
# Calculate Baseline (Kijun-sen)
historical_df['baseline'] = (historical_df['high'].rolling(window=26).max() + historical_df['low'].rolling(window=26).min()) / 2

# Calculate Turning Line (Tenkan-sen)
historical_df['turning_line'] = (historical_df['high'].rolling(window=9).max() + historical_df['low'].rolling(window=9).min()) / 2

# Calculate Lagging Line (Chikou Span)
historical_df['lagging_line'] = historical_df['close'].shift(26)

# Calculate Leading Span A (Senkou Span A)
historical_df['leading_span_A'] = ((historical_df['baseline'] + historical_df['turning_line']) / 2).shift(-26)

# Calculate Leading Span B (Senkou Span B)
historical_df['leading_span_B'] = (historical_df['high'].rolling(window=52).max() + historical_df['low'].rolling(window=52).min()) / 2
historical_df['leading_span_B'] = historical_df['leading_span_B'].shift(-26)

# Note: we don't drop NA values here because we want to preserve the full dataset for plotting

In [28]:
historical_df

,date,high,low,close,baseline,turning_line,lagging_line,leading_span_A,leading_span_B
0,2023-10-23,16.3400,15.2100,16.20,NaN,NaN,NaN,19.20000,NaN
1,2023-10-24,16.9300,16.3000,16.64,NaN,NaN,NaN,19.20000,NaN
2,2023-10-25,16.6300,15.5800,15.59,NaN,NaN,NaN,19.20000,NaN
3,2023-10-26,15.9100,15.0500,15.29,NaN,NaN,NaN,19.11250,NaN
4,2023-10-27,15.6000,14.9600,15.07,NaN,NaN,NaN,18.68250,NaN
5,2023-10-30,15.2900,14.5600,14.69,NaN,NaN,NaN,18.42250,NaN
6,2023-10-31,15.0100,14.5500,14.80,NaN,NaN,NaN,18.42250,NaN
7,2023-11-01,14.9600,14.4800,14.92,NaN,NaN,NaN,19.06500,NaN
8,2023-11-02,18.3000,17.2101,17.97,NaN,16.39000,NaN,19.06500,NaN
9,2023-11-03,19.0789,17.7600,18.89,NaN,16.77945,NaN,19.06000,NaN
